# ERAD

ERAD is a free, open-source Python toolkit for computing energy resilience measures in the face of hazards like earthquakes and flooding. It uses graph database to store data and perform computation at the household level for a variety of critical services that are connected by power distribution network. It uses asset fragility curves, which are functions that relate hazard severity to survival probability for power system assets including cables, transformers, substations, roof-mounted solar panels, etc. recommended in top literature. Programs like undergrounding, microgrid, and electricity backup units for critical infrastructures may all be evaluated using metrics and compared across different neighborhoods to assess their effects on energy resilience.

In [1]:
pip install NREL-erad

  Using cached pydantic-1.10.22-cp312-cp312-macosx_11_0_arm64.whl.metadata (154 kB)


  Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (61 kB)


Using cached pydantic-1.10.22-cp312-cp312-macosx_11_0_arm64.whl (2.5 MB)
Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl (13.7 MB)


  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.10.6
    Uninstalling pydantic-2.10.6:
      Successfully uninstalled pydantic-2.10.6


  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.5
    Uninstalling numpy-2.2.5:
      Successfully uninstalled numpy-2.2.5


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
infrasys 0.4.0 requires numpy<3,>=2, but you have numpy 1.26.4 which is incompatible.
infrasys 0.4.0 requires pydantic<3,>=2.7, but you have pydantic 1.10.22 which is incompatible.
grid-data-models 2.0.0 requires pydantic~=2.10.6, but you have pydantic 1.10.22 which is incompatible.


Note: you may need to restart the kernel to use updated packages.


We start by loading a sample GDM system using the gdmloader.

In [2]:
from gdm.distribution import DistributionSystem
from gdmloader.constants import GCS_CASE_SOURCE
from gdmloader.source import SystemLoader

gdm_loader = SystemLoader()
gdm_loader.add_source(GCS_CASE_SOURCE)

distribution_system: DistributionSystem = gdm_loader.load_dataset(
    source_name=GCS_CASE_SOURCE.name, 
    system_type=DistributionSystem, 
    dataset_name="p5r",
)
distribution_system.name = "p5r"

ImportError: cannot import name 'GetCoreSchemaHandler' from 'pydantic' (/opt/homebrew/Caskroom/miniconda/base/envs/gdm2/lib/python3.12/site-packages/pydantic/__init__.cpython-312-darwin.so)

ERAD Asset and Hazard systems are also infrasys systems. This means these class also have all the functionality to serialize and deserialize, add conconents and model validation. An `AssetSystem` can be build directly from a given GDM system using the `from_gdm` method. Once built, an instance of `HarzardSimulator` can be created by passing the asset system. Finally, the simualation can be run by calling the `run` method and passing an instance of `HazardSystem`. This system may  have definations of multiple hazards.

In [5]:
from erad.models.asset import Asset
from erad.runner import HarzardSimulator
from erad.systems.asset_system import AssetSystem
from erad.systems.hazard_system import HazardSystem

asset_system =  AssetSystem.from_gdm(distribution_system)
hazard_scenario = HarzardSimulator(asset_system=asset_system)
hazard_scenario.run(hazard_system=HazardSystem.earthquake_example())
for component in asset_system.iter_all_components():
    component.pprint()
    break

2025-05-01 18:17:28.394 | WARNING  | erad.runner:run:37 - No HazardFragilityCurves definations found in the passed HazardSystem using default curve definations


Asset(
    name='source',
    distribution_asset=UUID('9db89dca-068e-44aa-8338-39f63cdadb2d'),
    asset_type=<AssetTypes.substation: 0>,
    height=<Quantity(3, 'meter')>,
    latitude=36.60102672210813,
    longitude=-120.91292875147646,
    asset_state=[
        AssetState(
            name='',
            timestamp=datetime.datetime(2025, 5, 1, 18, 17, 28, 393778),
            wind_speed=None,
            flood_velocity=None,
            flood_depth=None,
            fire_boundary_dist=None,
            peak_ground_velocity=SpeedProbability(
                name='',
                survival_probability=0.5300401162709448,
                speed=<Quantity(43.7665689, 'centimeter / second')>
            ),
            peak_ground_acceleration=AccelerationProbability(
                name='',
                survival_probability=0.5027933876698256,
                acceleration=<Quantity(1.52785696, 'meter / second ** 2')>
            ),
            survival_probability=0.2665006656607766
        )
    ]
)